In [ ]:
import pandas as pd
import torch
from data.datasets import ClinicalCovid
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.pretraining import TabNetPretrainer
from augmentations import ClassificationSMOTE
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tabnet_utils import regularized_loss
import pickle

In [ ]:
normalization = False

# Preprocessing of train set

In [ ]:
initial_drop_list = ['ID', 'cough', 'sputum', 'chills', 'Sore throat',
                     'dizziness', 'stomachache', 'Diarrhea', 'Nausea',
                     'runny nose', 'Nasal congestion', 'alcohol']

train_set = ClinicalCovid('data/all_final.csv', drop_list=initial_drop_list)

train_dropped_list = train_set.drop_missing_columns(threshold=0.5)
train_set.fill_missing_data(n_dset=1, return_dset=0, iters=5, n_tree=50, print_kernel=False)
# train_set.drop_missing_row()
if normalization:
    train_set.normalize()

In [ ]:
train_set.dataframe.head()

In [ ]:
train_set.raw_frame.head()

In [ ]:
with open('imputed_data.pickle', 'wb') as h:
    pickle.dump(train_set, h)

In [ ]:
train_set.dataframe.head(10000)

# Pretrain Network

In [ ]:
print(train_set.x.shape)
print(train_set.y.shape)

In [ ]:
aucs = []
predictions = []
probabilities = []
ground_truth = []
for i in range(20):
    x_train, x_test, y_train, y_test = train_set.data_split(test_ratio=0.2, random_state=i)
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    print(f"x_train:{x_train.shape}, y_train:{y_train.shape}")
    print(f"x_test:{x_test.shape}, y_test:{y_test.shape}")
    unsupervised_model = TabNetPretrainer(
        device_name='cpu',
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=2e-2, weight_decay=0.0001),
        mask_type='entmax',  # "sparsemax"
        n_steps=1,
        seed=40
    )
    unsupervised_model.fit(
        X_train=x_train[0:1200],
        max_epochs=130,
        patience=100,
        eval_set=[x_train[1200:]],
        pretraining_ratio=0.8,
    )
    clf = TabNetClassifier(
        device_name='cpu',
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=2e-2),
        n_steps=2,
        scheduler_params={"step_size": 10,  # how to use learning rate scheduler
                          "gamma": 0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type='sparsemax',  # This will be overwritten if using pretrain model
        seed=40
    )

    clf.fit(
        X_train=x_train, y_train=y_train,
        eval_set=[(x_train, y_train), (x_test, y_test)],
        eval_name=['train', 'valid'],
        eval_metric=['balanced_accuracy', 'accuracy', 'auc'],
        weights=1,
        max_epochs=200,
        patience=100,
        from_unsupervised=unsupervised_model,
    )
    aucs.append(np.max(np.asarray(clf.history['valid_auc'])))
    preds = clf.predict(x_test)
    probs = clf.predict_proba(x_test)
    print(probs)
    predictions.append(preds)
    probabilities.append(probs)
    ground_truth.append(y_test)

In [ ]:
# aucs
# predictions
# probabilities
# ground_truth
print(np.mean(np.asarray(aucs)))
print(np.std(np.asarray(aucs)))

In [ ]:
predictions[0].shape

In [ ]:
from sklearn.metrics import f1_score, fbeta_score, precision_score, recall_score, balanced_accuracy_score

f1 = []
f2 = []
precisions = []
recalls = []
accuracies = []
for i, pred in enumerate(predictions):
    y = ground_truth[i]
    f1.append(f1_score(y, pred, pos_label=1, average='weighted'))
    f2.append(fbeta_score(y, pred, pos_label=1, beta=2, average='weighted'))
    precisions.append(precision_score(y, pred, pos_label=1, average='weighted'))
    recalls.append(recall_score(y, pred, pos_label=1, average='weighted'))
    accuracies.append(balanced_accuracy_score(y, pred))

print(np.mean(np.asarray(f1)), np.std(np.asarray(f1)))
print(np.mean(np.asarray(f2)), np.std(np.asarray(f2)))
print(np.mean(np.asarray(precisions)), np.std(np.asarray(precisions)))
print(np.mean(np.asarray(recalls)), np.std(np.asarray(recalls)))
print(np.mean(np.asarray(accuracies)), np.std(np.asarray(accuracies)))


In [ ]:

# x_categ = torch.randint(0, 5, (0, 0))     # category values, from 0 - max number of categories, in the order as passed into the constructor above
# x_cont = torch.randn(1, 10)               # assume continuous values are already normalized individually
# 

# pred = model(x_categ, x_cont) # (1, 1)

In [ ]:
from sklearn import metrics


def compute_auc(pred_logits, labels):
    logits = pred_logits.softmax(dim=1)
    logits = logits.detach().numpy()
    fpr, tpr, thresholds = metrics.roc_curve(labels, logits[:, 1], pos_label=1)
    return metrics.auc(fpr, tpr)


def plot_auc(pred_logits, labels):
    logits = pred_logits.softmax(dim=1)
    logits = logits.detach().numpy()
    fpr, tpr, thresholds = metrics.roc_curve(labels, logits[:, 1], pos_label=1)
    plt.plot(fpr,tpr)
    plt.savefig('auc_test.tif')


In [ ]:
import torch
import torch.nn as nn
from tab_transformer_pytorch import TabTransformer

aucs = []
predictions = []
ground_truth = []

for i in range(20):

    number_of_epochs = 4000
    x_train, x_test, y_train, y_test = train_set.data_split(test_ratio=0.2, random_state=i)
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    print(f"x_train:{x_train.shape}, y_train:{y_train.shape}")
    print(f"x_test:{x_test.shape}, y_test:{y_test.shape}")

    t_train = torch.tensor(x_train)
    t_test = torch.tensor(x_test)
    y_train = torch.tensor(y_train)
    y_train = y_train.type(torch.LongTensor)
    y_test = torch.tensor(y_test)
    y_test = y_test.type(torch.LongTensor)
    cat_train = torch.randint(0, 5, (1599, 0))
    cat_test = torch.randint(0, 5, (400, 0))
    print(t_train.shape)
    print(t_test.shape)
    print(cat_train.shape)

    model = TabTransformer(
        categories=(),  #10, 5, 6, 5, 8 tuple containing the number of unique values within each category
        num_continuous=110,  # number of continuous values
        dim=32,  # dimension, paper set at 32
        dim_out=2,  # binary prediction, but could be anything
        depth=6,  # depth, paper recommended 6
        heads=8,  # heads, paper recommends 8
        attn_dropout=0.1,  # post-attention dropout
        ff_dropout=0.1,  # feed forward dropout
        mlp_hidden_mults=(4, 2),  # relative multiples of each hidden dimension of the last mlp to logits
        mlp_act=nn.ReLU(),  # activation for final mlp, defaults to relu, but could be anything else (selu etc)
        # continuous_mean_std = cont_mean_std # (optional) - normalize the continuous values before layer norm
    )

    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(number_of_epochs):
        optimizer.zero_grad()
        model.train()
        logits = model(cat_train, t_train)
        # print(pred.shape)
        # print(y_train)
        loss = criterion(logits, y_train)
        loss.backward()
        optimizer.step()
        model.eval()
        logits = model(cat_test, t_test)
        auc = compute_auc(logits, y_test)
        print(epoch, loss.item(), auc)
    model.eval()
    logits = model(cat_test, t_test)
    auc = compute_auc(logits, y_test)
    aucs.append(auc)
    predictions.append(logits)
    ground_truth.append(y_test)


In [ ]:
model.eval()
logits = model(cat_test, t_test)
auc = plot_auc(logits, y_test)

In [ ]:
print(np.mean(np.asarray(aucs)))
print(np.std(np.asarray(aucs)))

In [ ]:
torch.argmax(predictions[0], dim=1)

In [ ]:
from sklearn.metrics import f1_score, fbeta_score, precision_score, recall_score, balanced_accuracy_score

f1 = []
f2 = []
precisions = []
recalls = []
accuracies = []
for i, pred in enumerate(predictions):
    pred = torch.argmax(pred, dim=1)
    y = ground_truth[i]
    f1.append(f1_score(y, pred, pos_label=1, average='weighted'))
    f2.append(fbeta_score(y, pred, pos_label=1, beta=2, average='weighted'))
    precisions.append(precision_score(y, pred, pos_label=1, average='weighted'))
    recalls.append(recall_score(y, pred, pos_label=1, average='weighted'))
    # print(recall_score(y, pred, pos_label=1, average='weighted'))
    # print(accuracy_score(y, pred))
    accuracies.append(balanced_accuracy_score(y, pred))

print(np.mean(np.asarray(f1)), np.std(np.asarray(f1)))
print(np.mean(np.asarray(f2)), np.std(np.asarray(f2)))
print(np.mean(np.asarray(precisions)), np.std(np.asarray(precisions)))
print(np.mean(np.asarray(recalls)), np.std(np.asarray(recalls)))
print(np.mean(np.asarray(accuracies)), np.std(np.asarray(accuracies)))

# Supervised Training

# Interpret

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
preds.shape

In [ ]:
print(preds)

In [ ]:
norm_vec = np.sum(np.exp(preds), axis=1).reshape(-1, 1)
print(norm_vec.shape)
norm_vec = np.tile(norm_vec, (1, 2))

p = np.exp(preds) / norm_vec

print(p)

In [ ]:
from sklearn import metrics

fpr, tpr, theasholds = metrics.roc_curve(y_test, p[:, 1])

In [ ]:
print(theasholds)

In [ ]:
from sklearn import metrics

fpr, tpr, _ = metrics.roc_curve(y_test, p[:, 1])

# with open('tabnet_fpr.pkl', 'wb') as f:
#     pickle.dump(fpr, f)
#
# with open('tabnet_tpr.pkl', 'wb') as f:
#     pickle.dump(tpr, f)

#create ROC curve
plt.plot(fpr, tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
explain_matrix, masks = clf.explain(x_test)

In [ ]:
masks[0].shape

In [ ]:
plt.plot(np.mean(explain_matrix, axis=0))
plt.show()

plt.plot(np.mean(masks[0], axis=0))
plt.show()

In [ ]:
from matplotlib.pyplot import figure

temp_mask = masks[0]

temp_mask[temp_mask > 0] = 1

sns.heatmap(temp_mask, cmap='BuGn', square=True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(18, 12))
my_colors = [(0.2, 0.3, 0.3), (0.4, 0.5, 0.4), (0.1, 0.7, 0), (0.1, 0.7, 0)]

sns.heatmap(temp_mask, cmap=my_colors, square=True, linecolor=(0.1, 0.2, 0.2), ax=ax)

# colorbar = ax.collections[0].colorbar
# M=temp_mask.max().max()
# colorbar.set_ticks([1/8*M,3/8*M,6/8*M])
# colorbar.set_ticklabels(['low','med','high'])

fig.show()
fig.savefig('Outputs3/total.pdf')

In [ ]:
preds = clf.predict(x_test)
norm_vec = np.sum(np.exp(preds), axis=1).reshape(-1, 1)
norm_vec = np.tile(norm_vec, (1, 2))
p = np.exp(preds) / norm_vec
preds = np.argmax(preds, axis=1)
print(preds.shape)
print(np.sum(preds == y_test) / y_test.shape)
print(confusion_matrix(y_test, preds))

In [ ]:
p_1 = np.where(preds == 1)[0]
r_1 = np.where(y_test == 1)[0]
p_0 = np.where(preds == 0)[0]
r_0 = np.where(y_test == 0)[0]

t_expire = p_1[np.in1d(p_1, r_1)]
f_expire = p_1[np.in1d(p_1, r_0)]

t_alive = p_0[np.in1d(p_0, r_0)]
f_alive = p_0[np.in1d(p_0, r_1)]
false_cases = np.where(y_test != preds)[0]
# false_cases = p_0[f_g]

In [ ]:
false_cases.shape

In [ ]:
explain_matrix, masks = clf.explain(x_test[t_expire])

plt.plot(np.sum(explain_matrix, axis=0))
plt.show()

plt.plot(np.sum(masks[0], axis=0))
plt.show()

# plt.plot(np.sum(masks[1], axis=0))
# plt.show()
#
# plt.plot(np.sum(masks[2], axis=0))
# plt.show()

temp = train_set.dataframe.columns.to_list()
temp.remove('outcome')
mask0 = pd.DataFrame(masks[0], columns=temp)

In [ ]:
mask0.head(1000)

In [ ]:
explain_matrix, masks = clf.explain(x_test[t_alive])

plt.plot(np.sum(explain_matrix, axis=0))
plt.show()

plt.plot(np.sum(masks[0], axis=0))
plt.show()

# plt.plot(np.sum(masks[1], axis=0))
# plt.show()
#
# plt.plot(np.sum(masks[2], axis=0))
# plt.show()

temp = train_set.dataframe.columns.to_list()
temp.remove('outcome')
mask0 = pd.DataFrame(masks[0], columns=temp)

# CODE FOR PAPER

In [ ]:
major_features_dict = dict()

In [ ]:
selected_group = t_expire  # true_alive ,true_expire, false_cases
group_name = 'true_expire'

threshold = 0.3

In [ ]:
import operator

explain_matrix, masks = clf.explain(x_test[selected_group])

mask = masks[0]

mask[mask > 0] = 1

mask = mask / mask.shape[0]

col_names_list = train_set.dataframe.columns.to_list()
col_names_list.remove('outcome')

selected_df = pd.DataFrame(x_test[selected_group], columns=col_names_list)

agg_mask = np.sum(mask, axis=0)
major_features = []
for i, col_name in enumerate(col_names_list):
    if agg_mask[i] < threshold:
        col_names_list[i] = ''
    else:
        major_features.append(col_name)

major_features_dict[group_name] = major_features

print(col_names_list)
print(major_features)

mask_agg = np.sum(mask, axis=0).tolist()

if group_name == 'true_alive':
    enumerate_object = enumerate(mask_agg)
    sorted_pairs = sorted(enumerate_object, key=operator.itemgetter(1))
    sorted_indices = [index for index, element in sorted_pairs]
    sorted_indices = list(reversed(sorted_indices))

mask_agg = np.asarray(mask_agg)

col_names_list = [col_names_list[i] for i in sorted_indices]
print(col_names_list)
mask_agg = mask_agg[sorted_indices]
mask = mask[:, sorted_indices]

fig, ax = plt.subplots()
ax.set_xticks(range(len(col_names_list)))
ax.set_xticklabels(col_names_list)
plt.xticks(fontsize=6, rotation=90)

ax.plot(mask_agg)
fig.savefig(f'Outputs3/{group_name}_agg.pdf')

temp = train_set.dataframe.columns.to_list()
temp.remove('outcome')

temp = [temp[i] for i in sorted_indices]

mask_pd = pd.DataFrame(mask, columns=temp)

In [ ]:
import matplotlib.pyplot as plt

# temp_mask = masks[0]
#
# temp_mask[temp_mask > 0] = 1

fig, ax = plt.subplots(1, 1, figsize=(10, 13))
my_colors = [(0.2, 0.3, 0.3), (0.4, 0.5, 0.4), (0.1, 0.7, 0), (0.1, 0.7, 0)]
ax.axes.get_yaxis().set_visible(False)
ax.set_xticks(range(len(col_names_list)))
ax.set_xticklabels(col_names_list)
# plt.xticks(fontsize=6, rotation=90)
# fig.axes[1].set_visible(False)

ax1 = sns.heatmap(mask[:20], cmap=my_colors, square=True, linewidths=0.01, cbar=False, linecolor=(0.1, 0.2, 0.2), ax=ax)
ax1.set_xticks(range(len(col_names_list)))
ax1.set_xticklabels(col_names_list, rotation=90, fontsize=6)
fig.savefig(f'Outputs3/{group_name}_heatmap.pdf')

In [ ]:
for index, row in mask_pd.iterrows():
    print(f"Index: {index}")
    for key in row.keys():
        if row[key] > 0:
            print('\t', key, selected_df.iloc[index][key])

In [ ]:
mask_pd.head(10)

In [ ]:
major_features_dict.keys()
venn_sets = []
venn_labels = []
for key in major_features_dict.keys():
    print(key, major_features_dict[key])
    venn_sets.append(set(major_features_dict[key]))
    venn_labels.append(key)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import venn3

venn3(venn_sets, venn_labels)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import venn3

plt.figure(figsize=(10, 10))

venn = venn3(venn_sets, venn_labels)

venn.get_label_by_id('100').set_text('\n'.join(venn_sets[0] - venn_sets[1] - venn_sets[2]))
# venn.get_label_by_id('110').set_text('\n'.join(venn_sets[0]&venn_sets[1]-venn_sets[2]))
# venn.get_label_by_id('010').set_text('\n'.join(venn_sets[1]-venn_sets[2]-venn_sets[0]))
venn.get_label_by_id('101').set_text('\n'.join(venn_sets[0] & venn_sets[2] - venn_sets[1]))
venn.get_label_by_id('111').set_text('\n'.join(venn_sets[0] & venn_sets[1] & venn_sets[2]))
venn.get_label_by_id('011').set_text('\n'.join(venn_sets[1] & venn_sets[2] - venn_sets[0]))
venn.get_label_by_id('001').set_text('\n'.join(venn_sets[2] - venn_sets[1] - venn_sets[0]))

plt.savefig(f'Outputs3/venn.pdf')

In [ ]:
features_union = set()
for key in major_features_dict.keys():
    print(key, major_features_dict[key])
    features_union = features_union.union(set(major_features_dict[key]))

print(list(features_union))

In [ ]:
cols = train_set.dataframe.columns.to_list()
cols.remove('outcome')

for feature in features_union:
    print(feature)

    group = x_test[t_alive]
    df = pd.DataFrame(group, columns=cols)
    sns.distplot(df[feature], hist=False, kde=True, label='True Alive')

    group = x_test[t_expire]
    df = pd.DataFrame(group, columns=cols)
    sns.distplot(df[feature], hist=False, kde=True, label='True Expire')

    group = x_test[false_cases]
    df = pd.DataFrame(group, columns=cols)
    sns.distplot(df[feature], hist=False, kde=True, label='False cases')

    # Plot formatting
    plt.legend(prop={'size': 12})
    plt.title(feature)
    plt.xlabel(feature)
    plt.ylabel('Density')
    plt.savefig(f"Outputs3/hist/{feature}.pdf")
    plt.show()

In [ ]:
correlation_mat = train_set.dataframe.corr()

plt.figure(figsize=(20, 20))

sns.heatmap(correlation_mat)

plt.savefig(f"Outputs3/correlation.pdf")

In [ ]:
correlation_mat['Headache']

In [ ]:
raw_data = test_set.__orig_dat__.iloc[t_expire]

In [ ]:
raw_data.head(10)